In [52]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)

Versão do TensorFlow:  2.16.1


In [53]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [54]:

# Carregar os dados do arquivo .pkl
with open('../../datas/horizontal_distance_datas.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
    
    
 

In [55]:
%load_ext tensorboard
log_dir = "../../logs/angle/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(2, activation='softmax')  #  
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=10000, validation_split=0.2, callbacks=[tensorboard_callback])


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9113 - loss: 1.5413 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 2/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9997 - loss: 0.0042 - val_accuracy: 0.9997 - val_loss: 0.0041
Epoch 3/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9996 - loss: 0.0043 - val_accuracy: 0.9990 - val_loss: 0.0042
Epoch 4/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9996 - loss: 0.0041 - val_accuracy: 0.9992 - val_loss: 0.0040
Epoch 5/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9994 - loss: 0.0040 - val_accuracy: 0.9984 - val_loss: 0.0043
Epoch 6/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9995 - loss: 0.0035 - val_accuracy: 0.9997 - val_loss: 0.0033
Epoch 7/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9996 - loss: 0.0033 - val_accuracy: 0.9987 - val_loss: 0.0037
Epoch 8/100
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9994 - loss: 0.0033 - val_accuracy: 0.9987 -

In [56]:
# Avaliar o modelo
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test))
print(f'Model Accuracy: {accuracy * 100:.2f}%')

9000/9000 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.9990 - loss: 0.0023
Model Accuracy: 99.90%


In [57]:
# Ações
# 0 Go to left
# 1 Go to Right

# GO TO RIGHT
targetX=100
agentX = 0
class_names = [  'Go to left', 'Go to Right', ]
predictions = model.predict(np.array([[targetX,agentX]]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO RIGHT: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)

# GO TO Left
targetX=0
agentX = 100
 
predictions = model.predict(np.array([[targetX,agentX]]))
 
score = tf.nn.softmax(predictions[0])
print(  "{}-Must be GO TO LETH: {} - with a {:.2f} percent confidence."
    .format(np.argmax(score),class_names[np.argmax(score)], 100 * np.max(score))
)
 



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1-Must be GO TO RIGHT: Go to Right - with a 73.11 percent confidence.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
0-Must be GO TO LETH: Go to left - with a 73.11 percent confidence.


In [58]:
%tensorboard

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

In [59]:
model.save("../../models/horizontal_direction_datas/model_saved.h5")
model.save(filepath="../../models/horizontal_direction_datas/model_saved.keras")


 

### From here to you only MACOS to test 

In [60]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('../../models/horizontal_direction_datas/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, '../../models/horizontal_direction_datas/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model '../../models/horizontal_direction_datas/saved_model/' ../../models/tf/horizontal_direction_datas/web_model

INFO:tensorflow:Assets written to: ../../models/horizontal_direction_datas/saved_model/assets


INFO:tensorflow:Assets written to: ../../models/horizontal_direction_datas/saved_model/assets
